In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import Dict, Text
import tensorflow_recommenders as tfrs

2022-12-06 20:28:07.156617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 20:28:07.258275: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 20:28:07.258294: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-06 20:28:07.766897: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
carts_df = pd.read_csv('all_carts.csv', index_col=0)
items_df = pd.read_csv('all_libelles.csv', index_col=0)

In [3]:
items_df = items_df[:100]
carts_df = carts_df[:100]

In [4]:
carts = tf.data.Dataset.from_tensor_slices(dict(carts_df))
items = tf.data.Dataset.from_tensor_slices(dict(items_df))

2022-12-06 20:28:11.600989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-06 20:28:11.601254: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 20:28:11.601302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-06 20:28:11.601335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-06 20:28:11.601364: W tensorf

In [5]:
carts_values = carts.map(lambda x:{'LIBELLE':x['LIBELLE'],'TICKET_ID':x['TICKET_ID']})
items_values = items.map(lambda x: x['LIBELLE'])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
cart_map = carts.map(lambda x: x['TICKET_ID'])
item_map = items.map(lambda x: x['LIBELLE'])

In [7]:
cart_id_vocabulary = tf.keras.layers.experimental.preprocessing.IntegerLookup(mask_token=None)
cart_id_vocabulary.adapt(cart_map)

In [8]:
libelles_vocabulary = tf.keras.layers.experimental.preprocessing.StringLookup(mask_token=None)
libelles_vocabulary.adapt(item_map)

In [10]:
class LibelleRecModel(tfrs.Model):
 
    def __init__(self,cart_model: tf.keras.Model,libelle_model: tf.keras.Model,task: tfrs.tasks.Retrieval):
        super().__init__()
        # Set up user and movie representations.
        self.cart_model = cart_model
        self.libelle_model = libelle_model
        # Set up a retrieval task.
        self.task = task
 
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.
        cart_embeddings = self.cart_model(features["TICKET_ID"])
        libelle_embeddings = self.libelle_model(features["LIBELLE"])
        return self.task(cart_embeddings, libelle_embeddings)

In [11]:
carts_model = tf.keras.Sequential([cart_id_vocabulary,tf.keras.layers.Embedding(cart_id_vocabulary.vocab_size(),64)])
libelles_model = tf.keras.Sequential([libelles_vocabulary,tf.keras.layers.Embedding(libelles_vocabulary.vocab_size(),64)])
 
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(items_values.batch(128).map(libelles_model)))

In [12]:
model = LibelleRecModel(carts_model,libelles_model,task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
model.fit(carts_values.batch(4096), epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.7800 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 460.4654 - regularization_loss: 0.0000e+00 - total_loss: 460.4654
Epoch 2/3
1/1 [==============================] - 0s 37ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0600 - factorized_top_k/top_10_categorical_accuracy: 0.2600 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 441.6501 - regularization_loss: 0.0000e+00 - total_loss: 441.6501
Epoch 3/3
1/1 [==============================] - 0s 37ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy:

In [13]:
recommends = tfrs.layers.factorized_top_k.BruteForce(model.cart_model)
recommends.index_from_dataset(items_values.batch(100).map(lambda libelle: (libelle, model.libelle_model(libelle))))

In [14]:
id_ = input('Enter the cart_id: ')
_, libelles = recommends(np.array([int(id_)]))
print('Top recommendation for cart',id_,libelles[0, :3])

Enter the cart_id: 35592159
Top recommendation for cart 35592159 tf.Tensor(
[b'CR JR PARF BIO.SPE AC.SENT.50ML' b'EAU MICELLAIRE 3 THES FL200ML'
 b'GD JDM4 TIARE FL 200ML'], shape=(3,), dtype=string)
